## Lako Grigalashvili


In [25]:
import numpy as np
import networkx as nx
import string
# Note changes; we need tools for manipulating labels
from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, Circle, Arrow, VeeHead, MultiLine # Possibly unnecessary
from bokeh.models.annotations import LabelSet
from bokeh.models.graphs import from_networkx
from bokeh.palettes import magma
import ipywidgets as widgets
from ipywidgets import Layout

App function creates widgets and takes arguments of Adjecency Matrix and Centrality measure 
Button on click function collects data given by user and calls centrality(Matrix,measure) function

In [26]:
def App():
    # create widgets
    cent = widgets.Dropdown(
                options=['degree','closeness', 'betweenness','eigenvector'],
                description='Centrality Measure',
                disabled=False,
            )

    matrix = widgets.Textarea(placeholder='accepted format: [[0,0,0]]', description='Matrix:',
               layout=Layout(width='50%', height='80px'))

    button = widgets.Button(description="Generate Graph")

    display(cent,matrix, button) 

    def on_button_clicked(b):
        
        adjMatrix = matrix.value
        measure = cent.value   
        centrality(eval(adjMatrix), measure) #eval converts string to actual array
        cent.layout.display = "none"
        matrix.layout.display = "none"
        button.layout.display = "none"
        
    button.on_click(on_button_clicked)

#### Centraliry function takes Adjecency Matrix and Method arguments
 
##### 1) 
Determines wether Graph is directed or not 
##### 2) 
Calculates cyrcles' radiuses
##### 3) 
returns Bokeh figure from PaintGraph function 

In [27]:
def centrality(matrix, method):  
  
    #Creating switcher function
    switcher = {
        "degree": nx.degree_centrality,
        "betweenness": nx.betweenness_centrality,
        "closeness": nx.closeness_centrality,
        "eigenvector": nx.eigenvector_centrality
    }
    
    directed = False 
    
    #If Graph is direct creatss Digraph of array. 
    if symmetric(matrix):
        H = nx.from_numpy_matrix(np.array(matrix))
    else:
        H = nx.DiGraph(np.array(matrix))
        directed = True
        
    func = switcher.get(method)  
    
    sizes = [] #Collects Centrality degrees
    dic = func(H)
    for i in dic.values():
        sizes.append(i)
    
    # paintGraph() takes arguments of Matrix(Adj. array), Centrality degrees([num]) and directed(boolean)         
    return paintGraph(H, sizes, directed) 
    
##Check if adjacency Matrix is symmetric. 
##Symmetric = Undirected Graph
def symmetric(matrix):
    
    for value in range(0,len(matrix)):
        if len(matrix) != len(matrix[value]):
            return False
    
    checkcolumn = 0
    list1 = []
    list2 = []
    
    while checkcolumn < len(matrix):
        list1+=matrix[checkcolumn]
        for row in matrix:
            list2.append(row[checkcolumn])
        if list1 != list2:
            return False
        else:
            checkcolumn+=1
    return True

### paintGraph Returns Plot
If Graph is directed adds Arrows

In [28]:
def paintGraph(H, sizes, directed):
    output_notebook()  
    
    #newSizes creates normalized data from sizes(centrality degrees) 
    newSizes = []

    for i in sizes:      
        if max(sizes)==0: 
            y = 0.2 #just to show cyrcle in case of all degrees are equal to 0
        else:
            y = (max(sizes)-min(sizes))/(max(sizes)-min(sizes)*(i-max(sizes))+max(sizes)) #general normalizing function
        
        newSizes.append(y)

        
    nodes = [] # Creating alphabetical nodes
    for x, y in zip(range(0, len(H.nodes)), string.ascii_uppercase):
        nodes.append(y)    
        
    plot = figure(title="Networkx Integration Demonstration", x_range=(-3,3), y_range=(-3,3),
                  tools="pan, box_zoom, reset, save", toolbar_location=None)

    graph = from_networkx(H, nx.spring_layout, scale=2.5, center=(0,0))

    pos = graph.layout_provider.graph_layout
    x,y=zip(*pos.values())
    
    #Creating labels with "Node, Centrality degree" format
    labels = list(zip(nodes, [ '%.2f' % elem for elem in sizes ]))
    
    source = ColumnDataSource({'x':x,'y':y,'kid':labels, 'radius':newSizes, 'color': magma(len(nodes))})   
   
    glyph = Circle(x="x", y="y", radius="radius", line_color="color", fill_color="color",line_width=0)
    plot.add_glyph(source, glyph)  
    
    labels = LabelSet(x='x', y='y', text='kid', source=source, text_color="black", background_fill_color="white",background_fill_alpha=0.8)
    plot.renderers.append(labels) 
    
    #if directed parameter is True, adds arrows to Graph
    if directed:
        for edges in H.edges:
            x0 = x[edges[0]]
            y0 = y[edges[0]]
            x1 = x[edges[1]]
            y1 = y[edges[1]]
            r1 = newSizes[edges[0]]
            r2 = newSizes[edges[1]]
            d = ((x1-x0)**2+(y1-y0)**2)**(1/2)
            
            #calculating starting and ending points for edges/arrow to start after cyrcle and finish before cyrcle 
            #for better visualizing
            plot.add_layout(Arrow(end=VeeHead(size=15), line_color="blue", 
                            x_start=(1-r1/d)*x0+r1/d*x1, 
                            y_start=(1-r1/d)*y0+r1/d*y1, 
                            x_end=(1-r2/d)*x1+r1/d*x0, 
                            y_end=(1-r2/d)*y1+r1/d*y0))
    else:
        
        plot.renderers.append(graph)
        
    plot.background_fill_color = "blue"
    plot.background_fill_alpha = 0.3  
    show(plot)


In [29]:
# examples:
#Matrix = [[0,1,1,1,0,0,0,0,0],[1,0,1,0,0,0,0,0,0],[1,1,0,1,0,0,0,0,0],[1,0,1,0,1,1,0,0,0],[0,0,0,1,0,1,1,1,0],[0,0,0,1,1,0,1,1,0],[0,0,0,0,1,1,0,1,1],[0,0,0,0,1,1,1,0,0],[0,0,0,0,0,0,1,0,0]]
#Matrix = [[0,0,1],[1,0,1],[1,1,0]]
#Matrix = [[0,1,0,1,1,0,1,0],[1,0,1,0,0,0,0,0],[0,1,0,0,1,0,1,1],[1,0,0,0,0,0,1,1],[1,0,1,0,0,0,1,0],[0,0,0,0,0,0,1,1],[1,0,1,1,1,1,0,0],[0,0,1,1,0,1,1,0]]


In [31]:
#You need to rerun App function in order to change centrality type
App()

Dropdown(description='Centrality Measure', options=('degree', 'closeness', 'betweenness', 'eigenvector'), valu…

Textarea(value='', description='Matrix:', layout=Layout(height='80px', width='50%'), placeholder='accepted for…

Button(description='Generate Graph', style=ButtonStyle())

Loading BokehJS ...